In [1]:
from functools import reduce
import pandas as pd
from datetime import datetime

import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.sql import DataFrame
from pyspark.sql import SparkSession
from pyspark.sql import Window

In [2]:
spark = SparkSession.builder.getOrCreate()

21/12/14 15:21:11 WARN Utils: Your hostname, emif-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.18 instead (on interface en0)
21/12/14 15:21:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/12/14 15:21:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/12/14 15:21:12 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
21/12/14 15:21:12 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
sdf = spark.read.json("/Users/emif/Downloads/test_read.txt")

In [4]:
sdf.printSchema()

root
 |-- brand_id: string (nullable = true)
 |-- carrier_damaged_units: long (nullable = true)
 |-- country_id: string (nullable = true)
 |-- customer_damaged_units: long (nullable = true)
 |-- defective_units: long (nullable = true)
 |-- distributor_damaged_units: double (nullable = true)
 |-- expired_units: long (nullable = true)
 |-- other_status_units: long (nullable = true)
 |-- p_snapshot_dt: string (nullable = true)
 |-- product_sku: string (nullable = true)
 |-- region_id: string (nullable = true)
 |-- sellable_units: long (nullable = true)
 |-- total_units: long (nullable = true)
 |-- warehouse_damaged_units: long (nullable = true)
 |-- warehouse_id: string (nullable = true)



In [10]:
expected_schema = T.StructType(
        [
            T.StructField("brand_id", T.StringType(), True),
            T.StructField("region_id", T.StringType(), True),
            T.StructField("country_id", T.StringType(), True),
            T.StructField("product_sku", T.StringType(), True),
            T.StructField("warehouse_id", T.StringType(), True),
            T.StructField("sellable_units", T.IntegerType(), True),
            T.StructField("defective_units", T.IntegerType(), True),
            T.StructField("expired_units", T.IntegerType(), True),
            T.StructField("customer_damaged_units", T.IntegerType(), True),
            T.StructField("distributor_damaged_units", T.IntegerType(), True),
            T.StructField("carrier_damaged_units", T.IntegerType(), True),
            T.StructField("warehouse_damaged_units", T.IntegerType(), True),
            T.StructField("other_status_units", T.IntegerType(), True),
            T.StructField("total_units", T.IntegerType(), True),
            T.StructField("p_snapshot_dt", T.DateType(), True),
        ]
    )

In [11]:
sdf_2 = spark.read.schema(expected_schema).json("/Users/emif/Downloads/test_read.txt")

In [12]:
sdf_2.printSchema()

root
 |-- brand_id: string (nullable = true)
 |-- region_id: string (nullable = true)
 |-- country_id: string (nullable = true)
 |-- product_sku: string (nullable = true)
 |-- warehouse_id: string (nullable = true)
 |-- sellable_units: integer (nullable = true)
 |-- defective_units: integer (nullable = true)
 |-- expired_units: integer (nullable = true)
 |-- customer_damaged_units: integer (nullable = true)
 |-- distributor_damaged_units: integer (nullable = true)
 |-- carrier_damaged_units: integer (nullable = true)
 |-- warehouse_damaged_units: integer (nullable = true)
 |-- other_status_units: integer (nullable = true)
 |-- total_units: integer (nullable = true)
 |-- p_snapshot_dt: date (nullable = true)



In [13]:
sdf_2.show(vertical=True)

-RECORD 0---------------------------------
 brand_id                  | BLACKRHINO   
 region_id                 | UK           
 country_id                | UK           
 product_sku               | GP-DFOM-807P 
 warehouse_id              | NCL1         
 sellable_units            | 68           
 defective_units           | 0            
 expired_units             | 0            
 customer_damaged_units    | 0            
 distributor_damaged_units | 0            
 carrier_damaged_units     | 0            
 warehouse_damaged_units   | 0            
 other_status_units        | 0            
 total_units               | 68           
 p_snapshot_dt             | 2021-12-07   
-RECORD 1---------------------------------
 brand_id                  | BLACKRHINO   
 region_id                 | UK           
 country_id                | UK           
 product_sku               | GP-DFOM-807P 
 warehouse_id              | NCL1         
 sellable_units            | 60           
 defective_